## Importing Required Libraries

In [ ]:
#import general packages
import os
import argparse
import logging
import sys
from typing import Dict, List
from dpu_utils.utils.richpath import RichPath
import pickle

import torch
from tqdm import tqdm
import pandas as pd

# Setting up local details:
# This should be the location of the checkout of the THEMAP repository:
repo_path = os.path.dirname(os.path.abspath(""))
CHECKOUT_PATH = repo_path
DATASET_PATH = os.path.join(repo_path, "datasets")

os.chdir(CHECKOUT_PATH)
sys.path.insert(0, CHECKOUT_PATH)

In [ ]:
# import visualization packages
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from sklearn.manifold import TSNE
import pandas as pd

light_color = plt.get_cmap("plasma").colors[170]
dark_color = "black"

#matplotlib.use("pgf")

matplotlib.rcParams.update(
    {
        "pgf.texsystem": "pdflatex",
        "font.family": "serif",
        "font.serif": "Computer Modern Roman",
        "font.size": 20,
        "text.usetex": True,
        "pgf.rcfonts": False,
    }
)

In [ ]:
# Load results from Meta-Learning experiments
AVAILABLE_EXP_NAME = ["FSMol_Eval_ProtoNet_2023-02-15_12-21-54",
                      "FSMol_Eval_MAML_2023-02-17_14-14-19",]
exp_name = "FSMol_Eval_ProtoNet_2023-02-15_12-21-54"
model = "ProtoNet"
evaluation_output_directory = os.path.join(CHECKOUT_PATH, "outputs", exp_name)
output_results = pd.read_csv(os.path.join(evaluation_output_directory, "summary", f"{model}_summary_num_train_requested_128.csv"))

## Analysis of Optimal Transport Distance Between Datasets

In [ ]:
available_otdd = ['test/hardness/otdd_Roberta-Zinc480M-102M.pkl',
                  'test/hardness/otdd_desc2D.pkl',
                  'test/hardness/otdd_gin_supervised_infomax.pkl',
                  'test/hardness/otdd_unimol.pkl',
                  'test/hardness/otdd_gin_supervised_contextpred.pkl',
                  'test/hardness/otdd_gin_supervised_masking.pkl',
]

In [ ]:
def internal_hardness(hardness_df, internal_hardness_path='test/hardness_intra/rf_64.pkl') -> pd.DataFrame:
    """
    Computes the internal hardness of the dataset.
    """  
    with open(os.path.join(DATASET_PATH, internal_hardness_path), 'rb') as f:
        test_tasks_hardness = pickle.load(f)

    weights=[]
    for chembl_id in hardness_df['assay']:
        weights.append(test_tasks_hardness[chembl_id][0].roc_auc)

    weights = torch.tensor(weights)
    hardness_df['internal_hardness'] = 1 - weights

    return hardness_df

In [ ]:
PATH_TO_OTDD = os.path.join(DATASET_PATH, 'test', 'hardness', 'otdd_gin_supervised_infomax.pkl')
PATH_TO_INTRA_HARDNESS = os.path.join(DATASET_PATH, 'train', 'hardness_intra', 'rf_16.pkl')

otdd_hardness_df = otdd_hardness(PATH_TO_OTDD, PATH_TO_INTRA_HARDNESS, k=10, train_tasks_weighted=True, weighting_method='rf')
otdd_hardness_df['hardness_norm'] = normalize(otdd_hardness_df['hardness']) 
# Visualizing the Hardness Distribution
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.hist(otdd_hardness_df['hardness'], bins=20, color=light_color, edgecolor=dark_color)
ax.set_title("Task Hardness Distribution")
ax.set_xlabel("Task Hardness")
ax.set_ylabel("Number of Tasks")

#ax.grid(True)
plt.show()
fig.savefig('assets/EXT_CHEM_hist.pdf', transparent=True, backend='pgf', bbox_inches='tight')
fig.savefig('assets/EXT_CHEM_hist.svg', transparent=True, bbox_inches='tight')

In [ ]:
otdd_hardness_df = internal_hardness(otdd_hardness_df, internal_hardness_path='test/hardness_intra/rf_16.pkl')
otdd_hardness_df['internal_hardness_norm'] = normalize(otdd_hardness_df['internal_hardness'])
x = pd.merge(output_results[['assay', 'delta_auprc', 'roc_auc']], otdd_hardness_df, on="assay")
print("Peasrson Inter-tasks hardness vs Performance measure", x['hardness_norm'].corr( x['roc_auc']))
print("Peasrson Intra-tasks hardness vs Performance measure", x['internal_hardness_norm'].corr( x['roc_auc']))
protein_hardness_df = pd.read_csv(os.path.join(DATASET_PATH, 'targets', 'esm2_output', 'esm2_t33_650M_UR50D', 'protein_hardness_10.0.csv'))
final_df = pd.merge(protein_hardness_df, x, on="assay")
final_df['inter_protein_hardness_norm'] = final_df['protein_hardness_mean_norm'] + final_df['hardness_norm']
final_df['inter_intra_protein_hardness_norm'] = final_df['inter_protein_hardness_norm'] + final_df['internal_hardness_norm']

In [ ]:
print(final_df['protein_hardness_mean'].corr( final_df['roc_auc']))
print(final_df['hardness'].corr( final_df['roc_auc']))
print(final_df['inter_protein_hardness_norm'].corr( final_df['roc_auc']))
print(final_df['inter_intra_protein_hardness_norm'].corr( final_df['roc_auc']))


fig, (ax0, ax1, ax2, ax3) = plt.subplots(4, 1, figsize=(10, 20))
sns.regplot(x="inter_intra_protein_hardness_norm", y="delta_auprc",data=final_df, ax=ax0)
sns.regplot(x="inter_protein_hardness_norm", y="delta_auprc", data=final_df, ax=ax1)
sns.regplot(x="protein_hardness_mean", y="delta_auprc", data=final_df, ax=ax2)
sns.regplot(x="hardness_norm", y="delta_auprc",data=final_df, ax=ax3)

ax0.set(xlabel='Chemical (intra and inter) and protein space hardness')
ax1.set(xlabel='Chemical (inter) and protein space')
ax2.set(xlabel='Protein space')
ax2.set(xlabel='Chemical space')
plt.show()

In [ ]:
print((final_df['hardness_norm'] + final_df['protein_hardness_mean_norm']).corr( final_df['roc_auc']))
print(final_df['hardness_norm'].corr( final_df['protein_hardness_mean_norm']))
print(final_df['hardness_norm'].corr( final_df['internal_hardness_norm']))

## Paper Figures

In [ ]:
print(final_df['hardness_norm'].corr( final_df['roc_auc']))
print(final_df['protein_hardness_mean_norm'].corr( final_df['roc_auc']))
print(final_df['inter_protein_hardness_norm'].corr( final_df['roc_auc']))
print(final_df['inter_intra_protein_hardness_norm'].corr( final_df['roc_auc']))

corr1 = round(final_df['hardness_norm'].corr( final_df['roc_auc']), 2)
corr2 = round(final_df['protein_hardness_mean_norm'].corr( final_df['roc_auc']), 2)
corr3 = round(final_df['inter_protein_hardness_norm'].corr( final_df['roc_auc']), 2)
corr4 = round(final_df['inter_intra_protein_hardness_norm'].corr( final_df['roc_auc']), 2)

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=(20, 12))
sns.regplot(x="hardness_norm", y="delta_auprc",data=final_df, ax=ax0)
sns.regplot(x="protein_hardness_mean_norm", y="delta_auprc", data=final_df, ax=ax1)
sns.regplot(x="inter_protein_hardness_norm", y="delta_auprc", data=final_df, ax=ax2)
sns.regplot(x="inter_intra_protein_hardness_norm", y="delta_auprc",data=final_df, ax=ax3)


ax0.set(xlabel='External chemical space hardness')
ax1.set(xlabel='External protein space hardness')
ax2.set(xlabel='External hardness (chemical and protein space)')
ax3.set(xlabel='External and internal hardness')

ax0.set(ylabel='$\Delta$ AUPRC')
ax1.set(ylabel='$\Delta$ AUPRC')
ax2.set(ylabel='$\Delta$ AUPRC')
ax3.set(ylabel='$\Delta$ AUPRC')

#ax0.legend(['Pearson r = {}'.format(corr1)], loc='upper right', fontsize=15, frameon=False)
#ax1.legend(['Pearson r = {}'.format(corr2)], loc='upper right', fontsize=15, frameon=False)
#ax2.legend(['Pearson r = {}'.format(corr3)], loc='upper right', fontsize=15, frameon=False)
#ax3.legend(['Pearson r = {}'.format(corr4)], loc='upper right', fontsize=15, frameon=False)

ax0.text(0.75, 0.9, 'Pearson r = {}'.format(corr1), bbox={
        'facecolor': 'grey', 'alpha': 0.5, 'pad': 10}, fontsize=15, transform=ax0.transAxes)
ax0.text(-0.1, 1.1, 'A', transform=ax0.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')


ax1.text(0.75, 0.9, 'Pearson r = {}'.format(corr2), bbox={
        'facecolor': 'grey', 'alpha': 0.5, 'pad': 10}, fontsize=15, transform=ax1.transAxes)
ax1.text(-0.1, 1.1, 'B', transform=ax1.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')


ax2.text(0.75, 0.9, 'Pearson r = {}'.format(corr3), bbox={
        'facecolor': 'grey', 'alpha': 0.5, 'pad': 10}, fontsize=15, transform=ax2.transAxes)
ax2.text(-0.1, 1.1, 'C', transform=ax2.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')


ax3.text(0.75, 0.9, 'Pearson r = {}'.format(corr4), bbox={
        'facecolor': 'grey', 'alpha': 0.5, 'pad': 10}, fontsize=15, transform=ax3.transAxes)
ax3.text(-0.1, 1.1, 'D', transform=ax3.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')


fig.tight_layout(pad=3.0)
#fig.savefig('assets/Hardness_vs_ProtoNet.pdf', transparent=True, backend='pgf', bbox_inches='tight')
fig.savefig('assets/Hardness_vs_ProtoNet.svg')
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(16, 6))
ax0.hist(final_df['hardness_norm'], bins=20, color=light_color, edgecolor=dark_color)
ax1.hist(final_df['protein_hardness_mean_norm'], bins=20, color=light_color, edgecolor=dark_color)

ax0.set_title("Task Hardness Distribution")
ax0.set_xlabel("External chemical space hardness")
ax0.set_ylabel("Number of tasks")
ax0.set_ylim([0, 35])
ax0.text(-0.1, 1.1, 'A', transform=ax0.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')

ax1.set_title("Task Hardness Distribution")
ax1.set_xlabel("External protein space hardness")
ax1.set_ylim([0, 35])
ax1.text(-0.1, 1.1, 'B', transform=ax1.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')



fig.savefig('assets/EXT_CHEM_EXT_PROT_hist.pdf', transparent=True, backend='pgf', bbox_inches='tight')
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(18, 6))
final_df.plot.scatter(x='hardness_norm', y='protein_hardness_mean_norm', c='delta_auprc', colormap='viridis', ax=ax0, s=50, alpha=0.8)

ax0.set_xlabel('External chemical hardness (normalized)')
ax0.set_ylabel('External protein hardness (normalized)')
ax0.grid(True)
ax0.axline((0, 0), slope=1, color='black', linestyle='--', linewidth=1)

corr5 = round(final_df['hardness_norm'].corr(final_df['protein_hardness_mean_norm']), 2)
ax0.text(0.65, 0.93, 'Pearson r = {}'.format(corr5), bbox={
        'facecolor': 'grey', 'alpha': 0.5, 'pad': 10}, transform=ax0.transAxes, fontsize=14)
ax0.text(-0.1, 1.1, 'A', transform=ax0.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')

#fig.savefig('assets/hardness_corr.svg', format='svg')

final_df.plot.scatter(x='hardness_norm', y='internal_hardness_norm', c='delta_auprc', colormap='viridis', ax=ax1, s=50, alpha=0.8)

ax1.set_xlabel('External chemical hardness (normalized)')
ax1.set_ylabel('Internal hardness (normalized)')
ax1.grid(True)
ax1.axline((0, 0), slope=1, color='black', linestyle='--', linewidth=1)

corr5 = round(final_df['hardness_norm'].corr(final_df['internal_hardness_norm']), 2)
ax1.text(0.65, 0.93, 'Pearson r = {}'.format(corr5), bbox={
        'facecolor': 'grey', 'alpha': 0.5, 'pad': 10}, transform=ax1.transAxes, fontsize=14)

#fig.savefig('assets/hardness_corr.svg', format='svg')
ax1.text(-0.1, 1.1, 'B', transform=ax1.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')

fig.savefig('assets/hardness_comparisson.pdf', transparent=True, backend='pgf', bbox_inches='tight')
plt.show()

In [ ]:
from functools import reduce

##Comparing different OTDD methods (for different feature types)
PATH_TO_OTDD1 = os.path.join(DATASET_PATH, 'test', 'hardness', 'otdd_gin_supervised_infomax.pkl')
PATH_TO_OTDD2 = os.path.join(DATASET_PATH, 'test', 'hardness', 'otdd_unimol.pkl')
PATH_TO_OTDD3 = os.path.join(DATASET_PATH, 'test', 'hardness', 'otdd_gin_supervised_masking.pkl')
PATH_TO_OTDD4 = os.path.join(DATASET_PATH, 'test', 'hardness', 'otdd_Roberta-Zinc480M-102M.pkl')
PATH_TO_OTDD5 = os.path.join(DATASET_PATH, 'test', 'hardness', 'otdd_desc2D.pkl')


PATH_TO_INTRA_HARDNESS = os.path.join(DATASET_PATH, 'train', 'hardness_intra', 'rf_16.pkl')


otdd_hardness_df1 = otdd_hardness(PATH_TO_OTDD1, PATH_TO_INTRA_HARDNESS, k=50, train_tasks_weighted=True, weighting_method='rf')
otdd_hardness_df2 = otdd_hardness(PATH_TO_OTDD2, PATH_TO_INTRA_HARDNESS, k=50, train_tasks_weighted=True, weighting_method='rf')
otdd_hardness_df3 = otdd_hardness(PATH_TO_OTDD3, PATH_TO_INTRA_HARDNESS, k=50, train_tasks_weighted=True, weighting_method='rf')
otdd_hardness_df4 = otdd_hardness(PATH_TO_OTDD4, PATH_TO_INTRA_HARDNESS, k=50, train_tasks_weighted=True, weighting_method='rf')
otdd_hardness_df5 = otdd_hardness(PATH_TO_OTDD5, PATH_TO_INTRA_HARDNESS, k=50, train_tasks_weighted=True, weighting_method='rf')


otdd_hardness_df = pd.merge(otdd_hardness_df1, otdd_hardness_df2, on='assay', how='outer', suffixes=('_Infomax', '_UniMol'))
otdd_hardness_df = pd.merge(otdd_hardness_df, otdd_hardness_df3, on='assay', how='outer', suffixes=('_UniMol', '_Masking'))
otdd_hardness_df = pd.merge(otdd_hardness_df, otdd_hardness_df4, on='assay', how='outer', suffixes=('_Masking', '_Roberta'))
otdd_hardness_df = pd.merge(otdd_hardness_df, otdd_hardness_df5, on='assay', how='outer', suffixes=('_Roberta', '_Desc2D'))

fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(2, 2, figsize=(20, 12))

fig.suptitle("External chemical hardness for different OTDD molecule features", fontsize=36)

ax0.scatter(otdd_hardness_df['hardness_Infomax'],  otdd_hardness_df['hardness_UniMol'], color=light_color, edgecolor=dark_color)
#ax0.set_title("Task hardness for differet OTDD molecule features")
ax0.set_xlabel("GIN Supervised Infomax")
ax0.set_ylabel("UniMol")
sp_corr = round(otdd_hardness_df['hardness_Infomax'].corr( otdd_hardness_df['hardness_UniMol'], method='spearman'), 2)
ax0.legend(['Spearman r = {}'.format(sp_corr)], loc='upper right', fontsize=20, frameon=False)
ax0.grid(True)

ax1.scatter(otdd_hardness_df['hardness_Infomax'],  otdd_hardness_df['hardness_Masking'], color=light_color, edgecolor=dark_color)
#ax1.set_title("Task hardness for differet OTDD molecule features")
ax1.set_xlabel("GIN Supervised Infomax")
ax1.set_ylabel("GIN Supervised Masking")
sp_corr = round(otdd_hardness_df['hardness_Infomax'].corr( otdd_hardness_df['hardness_Masking'], method='spearman'), 2)
ax1.legend(['Spearman r = {}'.format(sp_corr)], loc='upper right', fontsize=20, frameon=False)
ax1.grid(True)

ax2.scatter(otdd_hardness_df['hardness_Infomax'],  otdd_hardness_df['hardness_Roberta'], color=light_color, edgecolor=dark_color)
#ax2.set_title("Task hardness for differet OTDD molecule features")
ax2.set_xlabel("GIN Supervised Infomax")
ax2.set_ylabel("Roberta-Zinc 480M-102M")
sp_corr = round(otdd_hardness_df['hardness_Infomax'].corr( otdd_hardness_df['hardness_Roberta'], method='spearman'), 2)
ax2.legend(['Spearman r = {}'.format(sp_corr)], loc='upper right', fontsize=20, frameon=False)
ax2.grid(True)

ax3.scatter(otdd_hardness_df['hardness_Infomax'],  otdd_hardness_df['hardness'], color=light_color, edgecolor=dark_color)
#ax3.set_title("Task hardness for differet OTDD molecule features")
ax3.set_xlabel("GIN Supervised Infomax")
ax3.set_ylabel("desc2D")
sp_corr = round(otdd_hardness_df['hardness_Infomax'].corr( otdd_hardness_df['hardness'], method='spearman'), 2)
ax3.legend(['Spearman r = {}'.format(sp_corr)], loc='upper right', fontsize=20, frameon=False)
ax3.grid(True)

fig.tight_layout(pad=1.5)
#fig.savefig('assets/Hardness_vs_ProtoNet.pdf', transparent=True, backend='pgf', bbox_inches='tight')
plt.show()

In [ ]:
PATH_TO_OTDD = os.path.join(DATASET_PATH, 'test', 'hardness', 'otdd_gin_supervised_infomax.pkl')

k = [1, 10, 50, 100, 500, 1000, 2000, 4000]
corr_diff_k = []
for item in k:
    otdd_hardness_df = otdd_hardness(PATH_TO_OTDD, PATH_TO_INTRA_HARDNESS, k=item, train_tasks_weighted=True, weighting_method='rf')
    otdd_hardness_df['hardness_norm'] =normalize(otdd_hardness_df['hardness']) 
    m = pd.merge(output_results[['assay', 'delta_auprc', 'roc_auc']], otdd_hardness_df, on="assay")
    corr_diff_k.append(m['hardness_norm'].corr(m['roc_auc']))

print(corr_diff_k)
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.bar([str(i) for i in k], corr_diff_k)
ax.set_title('Pearson Correlation between EXT_CHEM', pad=20)
ax.set_xlabel('k (Number of nearest neighbors training tasks)')
ax.set_ylabel('Pearson Correlation')
#ax.text(-0.07, 1.05, 'B', transform=ax.transAxes,
#      fontsize=24, fontweight='bold', va='top', ha='right')

#ax.grid()
#ax.text(4000, -0.6, 'GIN Supervised Infomax', fontsize=15)
plt.show()

In [ ]:
from fs_mol.utils import compute_task_hardness_from_distance_matrix, normalize

DISTANCE_PATH = os.path.join(DATASET_PATH, 'targets', 'esm2_output', 'esm2_t36_3B_UR50D', 'protein_distance_matrix.pkl')
with open(DISTANCE_PATH, 'rb') as f:
    distance_matrix = pickle.load(f)

corr_ROC = []
corr_AUPRC = []
k_list = [1, 10, 50, 100, 500, 1000, 2000, 4000]
for k in k_list:
    hardness_protien = compute_task_hardness_from_distance_matrix(
        distance_matrix['distance_matrices'], aggr="mean_median", proportion=k
    )
    target_test_df = pd.read_csv(os.path.join(DATASET_PATH, "targets", "test_proteins.csv"))


    hardness_protien_mean_norm = normalize(hardness_protien[0])
    hardness_protien_median_norm = normalize(hardness_protien[1])

    protein_hardness_df = pd.DataFrame(
        {
            "protein_hardness_mean": hardness_protien[0],
            "protien_hardness_median": hardness_protien[1],
            "protein_hardness_mean_norm": hardness_protien_mean_norm,
            "protein_hardness_median_norm": hardness_protien_median_norm,
            "assay": distance_matrix['test_chembl_ids'],
        }
    )


    evaluation_output_directory = os.path.join(
        CHECKOUT_PATH, "outputs", "FSMol_Eval_ProtoNet_2023-02-15_12-21-54"
    )
    output_results = pd.read_csv(
        os.path.join(
            evaluation_output_directory, "summary", "ProtoNet_summary_num_train_requested_128.csv"
        )
    )

    df = protein_hardness_df.merge(output_results[["assay", "delta_auprc", "roc_auc"]], on="assay")
    print(
        "correlation between protein hardness and delta_auprc: ",
        df["protein_hardness_mean"].corr(df["delta_auprc"]),
    )
    corr_AUPRC.append(df["protein_hardness_mean"].corr(df["delta_auprc"]))
    print(
        "correlation between protein hardness and roc_auc: ",
        df["protein_hardness_mean"].corr(df["roc_auc"]),
    )
    corr_ROC.append(df["protein_hardness_mean"].corr(df["roc_auc"]))

fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.bar([str(i) for i in k_list], corr_ROC)
#ax.set_title('Correlation between external protein hardness and ROC-AUC', pad=20)
ax.set_title('ESM-t36-3B-UR50D', pad=20)
ax.set_xlabel('k (Number of nearest neighbors training tasks)')
ax.set_ylabel('Pearson Correlation')

ax.set_ylim(-0.5, 0)
ax.text(-0.07, 1.05, 'D', transform=ax.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')
#ax.grid()
#ax.text(4000, -0.6, 'GIN Supervised Infomax', fontsize=15)
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(16, 6))
ax0.bar([str(i) for i in k_list], corr_diff_k)
ax1.bar([str(i) for i in k_list], corr_ROC)


ax0.set_title("External chemical hardness", pad=20)
ax0.set_xlabel("k (number of nearest neighbors training tasks)")
ax0.set_ylabel("Number of tasks")
ax0.set_ylabel('Pearson correlation')

ax0.set_ylim([-0.65, 0.0])
ax0.text(-0.1, 1.05, 'A', transform=ax0.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')

ax1.set_title("External protein hardness", pad=20)
ax1.set_xlabel("k (number of nearest neighbors training tasks)")
ax1.set_ylim([-0.65, 0.0])
ax1.text(-0.1, 1.05, 'B', transform=ax1.transAxes,
      fontsize=24, fontweight='bold', va='top', ha='right')

#fig.suptitle('Correlation between external chemical hardness and ROC-AUC')

#fig.savefig('assets/hardness_dist.svg', format='svg')
plt.show()